In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('dataset.csv')

In [3]:
df.head(3)

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,...,-6.746,0,0.1430,0.0322,0.000001,0.358,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.166,...,-17.235,1,0.0763,0.9240,0.000006,0.101,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.359,...,-9.734,1,0.0557,0.2100,0.000000,0.117,0.120,76.332,4,acoustic


In [2]:
print(df.isna().sum().sort_values())

Unnamed: 0          0
tempo               0
valence             0
liveness            0
instrumentalness    0
acousticness        0
speechiness         0
mode                0
loudness            0
time_signature      0
key                 0
danceability        0
explicit            0
duration_ms         0
popularity          0
track_id            0
energy              0
track_genre         0
track_name          1
album_name          1
artists             1
dtype: int64


In [4]:
# Convert genre to a binary feature
df["track_genre"] = np.where(df["track_genre"] == "Rock", 1, 0)

print(df.isna().sum().sort_values())
print("Shape of the `music_df`: {}".format(df.shape))

Unnamed: 0          0
tempo               0
valence             0
liveness            0
instrumentalness    0
acousticness        0
speechiness         0
mode                0
loudness            0
time_signature      0
key                 0
danceability        0
explicit            0
duration_ms         0
popularity          0
track_id            0
energy              0
track_genre         0
track_name          1
album_name          1
artists             1
dtype: int64
Shape of the `music_df`: (114000, 21)
